### Основная гипотеза - рекомендовать похожие товары. Искать похожие товары непосредственно в том же каталоге, где находится и основной товар. Сравнивать (краткое описание + основные характеристики) товаров косинусной мерой, где мера похожести будет соответствовать весу рекомендуемого товара. 
Файл "ozon_test.txt" также должен лежать в каталоге пользователя по умолчанию

In [1]:
#### Да-да, куда без Спарка?

import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
# Подгружаем кучу либ, почти всё понадобится

import pyspark.sql.functions as f
from pyspark.sql.types import *
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, FloatType
import pandas as pd
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, FloatType
import pyspark.sql.functions as f
import re

In [3]:
# Загружаем основные данные для получения успешного резульата - описание и характеристики товаров магазина
data = spark.read.json('/labs/project02/item_details_full')
data=data.dropDuplicates()   # Удаляем дубликаты. Как ни странно, но они есть
data=data.fillna(' ')        # Заполняем пробелами пустые attr#, чтобы потом нормально склеить
data.cache()                 # На всякий случай кэшим (видимо здесь меня сейчас пнут по голове..)
data.registerTempTable('item_det')  # SQL рулит

In [4]:
# Кушаем все реквизиты кроме основного описания. Во-первых, оно большое, во-вторых ни о чём
item1 = spark.sql('''SELECT attr1||attr10||attr11||attr12||attr13||attr14||attr15||attr16||attr17||attr18||attr19||attr2||
                   attr20||attr21||attr22||attr23||attr24||attr25||attr26||attr27||attr28||attr29||attr3||attr30||attr31|| 
                   attr32||attr33||attr34||attr35||attr36||attr37||attr38||attr39||attr4||attr40||attr41||attr42||attr43||
                   attr44||attr45||attr46||attr47||attr48||attr49||attr5||attr50||attr51||attr52||attr53||attr54||attr55||
                   attr56||attr57||attr58||attr59||attr6||attr60||attr61||attr62||attr63||attr64||attr65||attr66||attr67||
                   attr68||attr69||attr7||attr70||attr71||attr72||attr73||attr74||attr75||attr76||attr77||attr8||attr9 as attr,
                   itemid,parent_id FROM item_det''')
item1.registerTempTable('item1')  # SQL рулит снова

In [5]:
# Загружаем второй основной файл - соответствие "id товара"-"id каталога"
schema_t = StructType(fields=[StructField("itemid",StringType()),StructField("catalogid",StringType())])
item_cat = spark.read.json("/labs/project02/catalogs",schema=schema_t)
item_cat.registerTempTable('catalogs')  # SQL рулит опять

In [6]:
# Скрестим сами товары с каталогами, к которым они относятся
item2 = spark.sql('''SELECT trim(a.attr) as attr,a.itemid,b.catalogid FROM item1 a, catalogs b where a.itemid=b.itemid''')
item2.registerTempTable('item2')  # SQL рулит обратно

In [7]:
# Загрузим тестовые товары
schema_test = StructType(fields=[StructField("item",StringType())])
item_test = spark.read.json("/labs/project02/ozon_test.txt",schema=schema_test)
item_test.registerTempTable('test')  # SQL рулит ещё раз

In [8]:
#Токенизация
@f.udf(ArrayType(StringType()))
def re_tokenizer(text):
    regex = re.compile(r'[\w\d]{2,}', re.U)
    return regex.findall(text.lower())

In [9]:
# Строим TF-IDF для всех товаров магазина
wordsData_udf = item2.withColumn('words', re_tokenizer('attr'))
tokenizer = Tokenizer(inputCol="attr", outputCol="words")
wordsData = tokenizer.transform(item2)
hashingTF = HashingTF(inputCol="words",
                      outputCol="TFFeatures",
                      numFeatures=10000, )
featurizedData = hashingTF.transform(wordsData_udf)
idf = IDF(inputCol="TFFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
from pyspark.ml.feature import Normalizer
t = Normalizer(inputCol='features', outputCol='norm_features', p=2.0)
normalizedData = t.transform(rescaledData)
# Уф.. Отмучались (рука бойца устала копипастить лабу 7)
normalizedData1=normalizedData.drop('attr','words','TFFeatures','features')  # Удаляем лишние колонки
normalizedData2=normalizedData1.cache()  # Вот здесь точно надо!
normalizedData2.registerTempTable('nd')  # Ну и ...

In [10]:
from pyspark.mllib.linalg import Vectors, Matrices
def cosine_similarity(a,b):
    dot = a.dot(b)
    return str(dot)

In [12]:
# Скрестим товары из теста с каталогами. 
# Их будет больше, чем просто товаров в тесте, так как многие товары имеют несколько каталогов
item_cat2= spark.sql('select * from catalogs where itemid in (SELECT item FROM test)') 
item_cat2.registerTempTable('item_cat2') # SQL


In [13]:
# Побежим по каталогам, заодно и не сломаемся, если у какого-то товара не будет каталога
test=spark.sql('select a.catalogid,count(a.catalogid) as cnt from item_cat2 a group by a.catalogid order by count(catalogid) desc')
test=test.toPandas()  # не успел понять, как бежать по SparkDF

In [20]:
from tqdm import tqdm
from pyspark.sql.types import FloatType

with open('project02.txt','w') as fil:
    for i in tqdm(range(test['cnt'].count())):  #бежим по каталогам. Бежать вне каталогов оочень дорого..
        if test['cnt'][i]==75:     #берем каталоги с 76 товарами (для теста). В боевых условиях закомментировать!
            nd = spark.sql('select * from nd where catalogid=%i'%int(test['catalogid'][i]))  #выбрали товары только по текущему каталогу          
            nd.cache()  # Здесь тоже точно нужно!
            nd.registerTempTable('courses') # Всё ещё SQL рулит
            item_cat3=item_cat2.filter('catalogid=%i'%int(test['catalogid'][i]))   #отберём товары теста текущей группы
            item_cat3=item_cat3.toPandas() # Ремарка пунктом выше
            for i in tqdm(range(item_cat3['itemid'].count())): #пробежимся по всем товарам теста текущей группы
                chosen_doc = spark.sql('SELECT * FROM courses WHERE itemid = %i' % int(item_cat3['itemid'][i])).first()
                # для каждого chosen_doc найдём косинусную меру товаров из его (товара из теста) группы
                sqlContext.registerFunction('cosine_similarity',
                    lambda x: cosine_similarity(x, chosen_doc.norm_features)
                )
                resultData = spark.sql(
                    '''
                    SELECT itemid, round(cosine_similarity(norm_features)*100000,2) as similarity 
                    FROM courses 
                    ORDER BY similarity DESC
                    '''
                )
                # возьмём столько значений, сколько нам надо и положим сразу в файл, а то всякое бывает, Spark же (не rdd же)
                pn={'item':item_cat3['itemid'][i],'recoms':{r.itemid:r.similarity for r in resultData[['itemid','similarity']].take(101)[0:100]}}
                fil.write(str(pn).replace("'",'"')+'\n')




  0%|          | 0/3571 [00:00<?, ?it/s]


  0%|          | 0/75 [00:00<?, ?it/s]


  1%|▏         | 1/75 [00:01<02:10,  1.76s/it]


  3%|▎         | 2/75 [00:03<02:08,  1.76s/it]


  4%|▍         | 3/75 [00:05<02:04,  1.72s/it]


  5%|▌         | 4/75 [00:06<01:59,  1.69s/it]


  7%|▋         | 5/75 [00:08<02:01,  1.73s/it]


  8%|▊         | 6/75 [00:10<02:00,  1.74s/it]


  9%|▉         | 7/75 [00:11<01:55,  1.70s/it]


 11%|█         | 8/75 [00:13<01:51,  1.67s/it]


 12%|█▏        | 9/75 [00:14<01:48,  1.64s/it]


 13%|█▎        | 10/75 [00:16<01:46,  1.64s/it]


 15%|█▍        | 11/75 [00:17<01:43,  1.61s/it]


 16%|█▌        | 12/75 [00:19<01:41,  1.61s/it]


 17%|█▋        | 13/75 [00:21<01:40,  1.63s/it]


 19%|█▊        | 14/75 [00:22<01:39,  1.62s/it]


 20%|██        | 15/75 [00:24<01:36,  1.61s/it]


 21%|██▏       | 16/75 [00:25<01:35,  1.61s/it]


 23%|██▎       | 17/75 [00:27<01:32,  1.60s/it]


 24%|██▍       | 18/75 [00:28<01:31,  1.60s/it]


 25%|██▌       | 19/75 

In [ ]:
# Дальше чёрная магия. Рисовал alexey.sotnikov (я не понял, как, но РАБОТАЕТ отлично!)
# Данный блок объединяет строки с одинаковым 'item' (такое возникает из-за принадлежности одного товара
# разным категориям), склеивая рекомендации, и оставляет сто одну самую похожую

import json
itemrecs={}
with open('project02.txt','r') as fil:
    for line in fil:
        data = json.loads(line)
        item = data['item']
        tmrs = data['recoms']
        if itemrecs.get(item) is None :
            itemrecs[item] = tmrs
        else :
            for i in tmrs :
                if itemrecs[item].get(i) is None :
                    itemrecs[item][i] = tmrs[i]
                elif itemrecs[item][i] < tmrs[i] :
                    itemrecs[item][i] = tmrs[i]
s=''
with open('project02.001','w') as fil:
    for items in itemrecs:
        s = '{"item": "' + str(items) +'", "recoms": {'
        sortedlist = sorted(itemrecs[items].items(), key = lambda x: float(x[1]), reverse = True)
        if len(sortedlist)>101 :
            sortedlist = sortedlist[0:101]
        for i in range(len(sortedlist)):
            if i == 0 :
                delimiter = ''
            else :
                delimiter = ', '
            s = s + delimiter + '"' + sortedlist[i][0] + '": ' + str(sortedlist[i][1])
        s = s + '}}'
        fil.write(s+'\n')


In [ ]:
# Данный блок склеивает полученный файл с тестом по item, удаляет первую похожесть (на саму себя со значением 100000) 
# и добавляет к непосчитанным строкам похожесть - саму себя со значеним 99999

itemrecs={}
with open('project02.001','r') as fil:
    for line in fil:
        data = json.loads(line)
        item = data['item']
        tmrs = data['recoms']
        if itemrecs.get(item) is None :
            itemrecs[item] = tmrs
        else :
            for i in tmrs :
                if itemrecs[item].get(i) is None :
                    itemrecs[item][i] = tmrs[i]
                elif itemrecs[item][i] < tmrs[i] :
                    itemrecs[item][i] = tmrs[i]

with open('project02.txt', 'w') as f_end:
    with open('ozon_test.txt', 'r') as f:
        for line in f:
            data = json.loads(line)
            item = data['item']
            recoms = {}
            if itemrecs.get(item) is not None :
                recoms = itemrecs[item]
                recoms.pop(item, None)
                sortedlist = sorted(recoms.items(), key = lambda x: float(x[1]), reverse = True)
                s = '{"item": "' + str(item) +'", "recoms": {'
                for i in range(len(sortedlist)):
                    if i == 0 :
                        delimiter = ''
                    else :
                        delimiter = ', '
                    s = s + delimiter + '"' + sortedlist[i][0] + '": ' + str(sortedlist[i][1])
                s = s + '}}'
            else :
                s = '{"item": "' + str(item) +'", "recoms": {"' + str(item) + '": 99999.99}}'
            f_end.write(s+'\n')


In [ ]:
spark.stop()  # "Не забудьте освободить ресурсы" (c)

### С чем пришлось столкнуться:
#### 1) в лоб решать задачу нереально, предварительный прогноз расчета - больше месяца
#### 2) нашлись три интересных элемента в тесте. У двух нет ссылки на каталог (но здесь можно хотя бы поискать похожести на всём списке товаров), а один вообще отсутствует в списке товаров. Ни описания, ничего :)
#### 2) попытки понять точнее, как работает ozon, особым успехом не увенчались, так как каталог практически полностью обновился. Если 5 лет назад предлагали книги к каким-то товарам, то теперь это DVD, как минимум и т.д. Да и алгоритмы у них уже сильно поменялись. По товару из хака ни одна рекомендация на сайте не добавляет микробаллов :)